# Tests of the LSTM network to detect Slips behavioral letters

In [2]:
import sys
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from random import shuffle

#import lstm_dataset_functions
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Masking, Dropout
from keras.layers.recurrent import GRU
from keras.callbacks import ModelCheckpoint

In [59]:
dataset_file='datasets/all_datasets_raw'
with open(dataset_file, 'rb') as csvfile:
    df = pd.read_csv(csvfile, delimiter='|', names=[ "note", "label", "model_id", "state"], skipinitialspace=True)

In [60]:
# Clean the dataset
df.dropna(axis=0, how='any', inplace=True)

In [24]:
df.describe(include='all')

,note,label,model_id,state
count,7616,7555,7616,7574
unique,7555,7462,6459,2117
top,[ ],From-Botnet-TCP-Unknown--24 ...,[36m147.32.84.170-209.85.148.104-80-tcp[0m ...,2
freq,61,2,3,2203


In [25]:
def filter_by_string(df, col, string):
    ''' A function to filter the columns and rows'''
    return df[df[col].str.contains(string, regex=False) == True]

In [84]:
# Add a new column to the dataframe with the label. The label is 'Normal' for the normal data and 'Malcious' for the malware data
df.loc[df.label.str.contains('Normal'),'label'] = 'Normal'
df.loc[df.label.str.contains('Botnet'),'label'] = 'Malicious'
df.loc[df.label.str.contains('Malware'),'label'] = 'Malicious'

In [86]:
print('Amount of normal outtupless: {}. Amount of botnet outtuples: {}'.format(len(normal_data), len(botnet_data)))

Amount of normal outtupless: 1294. Amount of botnet outtuples: 6213


In [98]:
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True)

In [99]:
test_data

,note,label,model_id,state
4848,[780],Normal,[36m147.32.84.170-209.85.148.104-443-tcp[0m ...,99.z.i.i.i.0ZZ.I.i.i.i.0Z0zZ.Z.Z.Z.z.
639,[4803],Malicious,[36m147.32.84.165-120.138.121.194-6667-tcp[0...,2
7539,[7557],Normal,[36m147.32.84.170-209.85.149.100-80-tcp[0m ...,9
6629,[6953],Malicious,[36m147.32.84.165-77.238.167.32-80-tcp[0m ...,"88*0YY+y*Y,Y*"
6946,[7026],Normal,[36m147.32.84.170-74.125.232.205-80-tcp[0m ...,99z*I*i*
1091,[4870],Malicious,[36m147.32.84.165-186.92.63.171-6667-tcp[0m ...,22*000S
5489,[6442],Malicious,[36m147.32.84.165-178.137.128.94-6667-tcp[0m...,22*
3259,[4268],Malicious,[36m147.32.84.165-66.39.2.13-25-tcp[0m ...,2
5383,[5391],Malicious,[36m147.32.84.165-151.151.26.152-25-tcp[0m ...,22*
7154,[7292],Normal,[36m147.32.84.134-50.16.227.193-80-tcp[0m ...,96.
